# Create AI-Tutor vector database

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import chromadb

# create client and a new collection
# chromadb.EphemeralClient saves data in-memory.
chroma_client = chromadb.PersistentClient(path="./ai-tutor-db")
chroma_collection = chroma_client.create_collection("ai-tutor-db")

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Define a storage context object using the created vector store.
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
import json
from llama_index.core.schema import TextNode

def load_jsonl_create_nodes(filepath):
    nodes = []  # List to hold the created node objects
    with open(filepath, "r") as file:
        for line in file:
            # Load each line as a JSON object
            json_obj = json.loads(line)
            # Extract required information
            title = json_obj.get("title")
            url = json_obj.get("url")
            content = json_obj.get("content")
            source = json_obj.get("source")
            # Create a TextNode object and append to the list
            node = TextNode(
                text=content,
                metadata={"title": title, "url": url, "source": source},
                excluded_embed_metadata_keys=["title", "url", "source"],
                excluded_llm_metadata_keys=["title", "url", "source"],
            )
            nodes.append(node)
    return nodes

In [ ]:
filepath = "../combined_data.jsonl"
nodes = load_jsonl_create_nodes(filepath)

print(f"Loaded {len(nodes)} nodes/chunks from the JSONL file\n ")

node = nodes[0]
print(f"ID: {node.id_} \nText: {node.text}, \nMetadata: {node.metadata}")

print("\n")

node = nodes[-10000]
print(f"ID: {node.id_} \nText: {node.text}, \nMetadata: {node.metadata}")

In [ ]:
# # Create the pipeline to apply the transformation on each chunk,
# # and store the transformed text in the chroma vector store.
# pipeline = IngestionPipeline(
#     transformations=[
#         text_splitter,
#         QuestionsAnsweredExtractor(questions=3, llm=llm),
#         SummaryExtractor(summaries=["prev", "self"], llm=llm),
#         KeywordExtractor(keywords=10, llm=llm),
#         OpenAIEmbedding(),
#     ],
#     vector_store=vector_store
# )

# nodes = pipeline.run(documents=documents, show_progress=True);

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex

# embeds = OpenAIEmbedding(model="text-embedding-3-small", mode="similarity")
# embeds = OpenAIEmbedding(model="text-embedding-3-large", mode="similarity")
embeds = OpenAIEmbedding(model="text-embedding-3-large", mode="text_search")
# embeds = OpenAIEmbedding(model="text-embedding-ada-002", mode="similarity")

# Build index / generate embeddings using OpenAI.
index = VectorStoreIndex(nodes=nodes, show_progress=True, use_async=True, storage_context=storage_context, embed_model=embeds, insert_batch_size=3000,)

In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(temperature=0, model="gpt-3.5-turbo", max_tokens=None)
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5, embed_model=embeds)

In [ ]:
res = query_engine.query("What is the LLaMa model?")

In [ ]:
res.response

In [ ]:
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Title\t", src.metadata['title'])
  print("Text\t", src.text)
  print("Score\t", src.score)
  print("Metadata\t", src.metadata) 
  print("-_"*20)

# Load DB from disk

In [1]:
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
# Create your index
db2 = chromadb.PersistentClient(path="./ai-tutor-db")
chroma_collection = db2.get_or_create_collection("ai-tutor-db")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:chromadb.api.segment:Collection ai-tutor-db is not created.


In [2]:
# Create your index
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [3]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.vector_stores import (
    ExactMatchFilter,
    MetadataFilters,
    MetadataFilter,
    FilterOperator,
    FilterCondition,
)

filters = MetadataFilters(
    filters=[
        MetadataFilter(key="source", value="lanchain_course"),
        MetadataFilter(key="source", value="langchain_docs"),
    ],
    condition=FilterCondition.OR,
)

llm = OpenAI(temperature=0, model="gpt-3.5-turbo", max_tokens=None)
embeds = OpenAIEmbedding(model="text-embedding-3-large", mode="text_search")
# query_engine = index.as_query_engine(
#     llm=llm, similarity_top_k=5, embed_model=embeds, verbose=True, streaming=True, filters=filters
# )
query_engine = index.as_query_engine(
    llm=llm, similarity_top_k=5, embed_model=embeds, verbose=True,
)

In [4]:
res = query_engine.query("What is the LLama model?")

# history = ""   
# for token in res.response_gen:
#     history += token
#     print(history)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [5]:
res.response

'The LLama model is a family of large language models (LLMs) released by Meta AI, with different model sizes ranging from 7 billion to 65 billion parameters in the first version. The developers reported that the 13 billion parameter model outperformed larger models like GPT-3 and was competitive with state-of-the-art models like PaLM and Chinchilla. The model weights were released to the research community under a noncommercial license, but they were leaked to the public shortly after.'

In [6]:
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Source\t", src.metadata['source'])
  print("Title\t", src.metadata['title'])
  print("Text\t", src.text)
  print("Score\t", src.score)
  print("-_"*20)

Node ID	 6f8c13d8-1458-444c-857b-c1dc11d7f134
Source	 wikipedia
Title	 LLaMA
Text	 LLaMA LLaMA (Large Language Model Meta AI) is a family of large language models (LLMs), released by Meta AI starting in February 2023. For the first version of LLaMa, four model sizes were trained: 7, 13, 33 and 65 billion parameters. LLaMA's developers reported that the 13B parameter model's performance on most NLP benchmarks exceeded that of the much larger GPT-3 (with 175B parameters) and that the largest model was competitive with state of the art models such as PaLM and Chinchilla. Whereas the most powerful LLMs have generally been accessible only through limited APIs (if at all), Meta released LLaMA's model weights to the research community under a noncommercial license. Within a week of LLaMA's release, its weights were leaked to the public on 4chan via BitTorrent. In July 2023, Meta released several models as Llama 2, using 7, 13 and 70 billion parameters.
Score	 0.5133216393307418
-_-_-_-_-_-_-_

In [ ]:
from IPython.display import Markdown, display
# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

In [ ]:
prompts_dict = query_engine.get_prompts()

In [ ]:
display_prompt_dict(prompts_dict)